In [1]:

from json import loads
from dotenv import load_dotenv
from graphrag_sdk.orchestrator import Orchestrator
from graphrag_sdk.agents.kg_agent import KGAgent
from graphrag_sdk.models.openai import OpenAiGenerativeModel
from graphrag_sdk import (
    Ontology, Entity, Relation, Attribute, AttributeType, KnowledgeGraph, KnowledgeGraphModelConfig
)

# Load environment variables
load_dotenv()

True

Reads data from JSON files into two Knowledge Graphs, representing multi-agent entities and their relationships.

In [2]:
def import_data():
        with open("data/cities.json") as f:
            cities = loads(f.read())
        with open("data/restaurants.json") as f:
            restaurants = loads(f.read())
        with open("data/attractions.json") as f:
            attractions = loads(f.read())

        for city in cities:
            restaurants_kg.add_node(
                "City",
                {
                    "name": city["name"],
                    "weather": city["weather"],
                    "population": city["population"],
                },
            )
            restaurants_kg.add_node("Country", {"name": city["country"]})
            restaurants_kg.add_edge(
                "IN_COUNTRY",
                "City",
                "Country",
                {"name": city["name"]},
                {"name": city["country"]},
            )

            attractions_kg.add_node(
                "City",
                {
                    "name": city["name"],
                    "weather": city["weather"],
                    "population": city["population"],
                },
            )
            attractions_kg.add_node("Country", {"name": city["country"]})
            attractions_kg.add_edge(
                "IN_COUNTRY",
                "City",
                "Country",
                {"name": city["name"]},
                {"name": city["country"]},
            )

        for restaurant in restaurants:
            restaurants_kg.add_node(
                "Restaurant",
                {
                    "name": restaurant["name"],
                    "description": restaurant["description"],
                    "rating": restaurant["rating"],
                    "food_type": restaurant["food_type"],
                },
            )
            restaurants_kg.add_edge(
                "IN_CITY",
                "Restaurant",
                "City",
                {"name": restaurant["name"]},
                {"name": restaurant["city"]},
            )

        for attraction in attractions:
            attractions_kg.add_node(
                "Attraction",
                {
                    "name": attraction["name"],
                    "description": attraction["description"],
                    "type": attraction["type"],
                },
            )
            attractions_kg.add_edge(
                "IN_CITY",
                "Attraction",
                "City",
                {"name": attraction["name"]},
                {"name": attraction["city"]},
            )

Creating Ontology.

In [3]:
restaurants_ontology = Ontology()
attractions_ontology = Ontology()
restaurants_ontology.add_entity(
    Entity(
        label="Country",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
        ],
    )
)
restaurants_ontology.add_entity(
    Entity(
        label="City",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="weather",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="population",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
        ],
    )
)
restaurants_ontology.add_entity(
    Entity(
        label="Restaurant",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="description",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="rating",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
            Attribute(
                name="food_type",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
        ],
    )
)
restaurants_ontology.add_relation(
    Relation(
        label="IN_COUNTRY",
        source="City",
        target="Country",
    )
)
restaurants_ontology.add_relation(
    Relation(
        label="IN_CITY",
        source="Restaurant",
        target="City",
    )
)

attractions_ontology.add_entity(
    Entity(
        label="Country",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
        ],
    )
)
attractions_ontology.add_entity(
    Entity(
        label="City",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="weather",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="population",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
        ],
    )
)
attractions_ontology.add_entity(
    Entity(
        label="Attraction",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="description",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="type",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
        ],
    )
)
attractions_ontology.add_relation(
    Relation(
        label="IN_COUNTRY",
        source="City",
        target="Country",
    )
)
attractions_ontology.add_relation(
    Relation(
        label="IN_CITY",
        source="Attraction",
        target="City",
    )
)

Multi-agent objects.

In [4]:
# Create Knowledge Graphs
model = OpenAiGenerativeModel("gpt-4o")
restaurants_kg = KnowledgeGraph(
    name="restaurants",
    ontology=restaurants_ontology,
    model_config=KnowledgeGraphModelConfig.with_model(model),
)
attractions_kg = KnowledgeGraph(
    name="attractions",
    ontology=attractions_ontology,
    model_config=KnowledgeGraphModelConfig.with_model(model),
)

# Import data into Knowledge Graphs
import_data()

# Setup agents
restaurants_agent = KGAgent(
    agent_id="restaurants_agent",
    kg=restaurants_kg,
    introduction="I'm a restaurant agent, specialized in finding the best restaurants for you.",
)

attractions_agent = KGAgent(
    agent_id="attractions_agent",
    kg=attractions_kg,
    introduction="I'm an attractions agent, specialized in finding the best attractions for you.",
)

Orchestrator and multi-agent questioning.

In [5]:

# Initialize the orchestrator and register agents
orchestrator = Orchestrator(
    model,
    backstory="You are a trip planner, and you want to provide the best possible itinerary for your clients.",
)
orchestrator.register_agent(restaurants_agent)
orchestrator.register_agent(attractions_agent)

# Query the orchestrator
runner = orchestrator.ask("Write me a 2 day itinerary for a trip to Italy. Do not ask any questions to me, just provide your best itinerary.")

In [6]:
print(runner.output)

Thank you for your patience. Based on the information gathered from our agents, here's a detailed and culturally enriching 2-day itinerary for your trip to Italy:

### Day 1: Venice and Florence
**Morning: Venice Exploration**
- **Piazza San Marco:** Start your day by visiting Venice's most famous square, known for its stunning architecture and historical significance.
- **Ristorante Quadri:** Enjoy a gourmet lunch at this historic restaurant located right in Piazza San Marco. Relish exquisite Italian cuisine while soaking in the ambiance of Venice. *Rating: 4.8*

**Afternoon: Historic Venice**
- **St. Mark’s Basilica:** Explore this magnificent cathedral with its lavish design and breathtaking mosaics.
- **Gondola Ride:** Experience the quintessential Venetian activity by taking a scenic gondola ride through the canals.

**Evening: Travel to Florence**
- Take an early evening train from Venice to Florence.
- Check into your hotel in Florence and take a leisurely evening stroll around 